In [20]:
import praw
import pandas as pd
import datetime
import time

In [2]:
reddit = praw.Reddit('bot1') # config in ~/.config/praw.ini

In [3]:
def scan_thread(submission):
    # scan a thread to read all submissions/comments
    
    # add the submission itself
    if submission.author is None:
        ds_sub = pd.DataFrame()
    else:
        ds_sub = pd.DataFrame(data = [[submission.id, submission.title, submission.author.name,
                                      submission.author_flair_text,
                                      datetime.datetime.fromtimestamp(submission.created_utc),
                                      submission.selftext]],
                              columns = ['id', 'title', 'author', 'flair', 'created', 'text']);
    
    # now go through the comments
    ds = pd.DataFrame()
    submission.comments.replace_more(limit=None)
    comment_queue = submission.comments[:]  # Seed with top-level
    for comment in comment_queue:
        if comment.author is None:
            continue
        entry = pd.DataFrame(data = [[comment.id, comment.author.name, comment.author_flair_text,
                                     datetime.datetime.fromtimestamp(comment.created_utc),
                                     comment.body, comment.parent_id]],
                             columns = ['id', 'author', 'flair', 'created', 'text', 'parent']);
        ds = ds.append(entry, ignore_index = True)
            
    return ds_sub, ds

In [4]:
subs = pd.read_csv('nba_submissions_reg18.csv', usecols = ['id'])

In [5]:
ds_sub = pd.DataFrame()
ds = pd.DataFrame()
#fsub_count = 1
#fcom_count = 1
#sub_count = 1

In [29]:
ds_sub = pd.DataFrame()
ds = pd.DataFrame()
fsub_count = 4
fcom_count = 4
sub_count = 1

In [ ]:
skip = True
tstart = time.time()
for sub_id in subs['id']:
    if sub_count%1000 == 1:
        print(sub_count)
        tcur = time.time()
        print(round((tcur-tstart)/60, 2))
        tstart = tcur
    sub_count += 1
    
    if skip:
        if sub_id == 'a0o2iy':
            skip = False
        else:
            continue
    
    sub = reddit.submission(id=sub_id)
    single_sub, single_ds = scan_thread(sub)
    ds_sub = ds_sub.append(single_sub, ignore_index = True)
    ds = ds.append(single_ds, ignore_index = True)
    if ds.shape[0] > 200000:
        fname = 'nba_comments_reg18_'+ str(fcom_count) + '.csv'
        print(fname)
        ds.to_csv(fname)
        fcom_count += 1
        ds = pd.DataFrame()
    if ds_sub.shape[0] > 200000:
        fname = 'nba_submissions_reg18_' + str(fsub_count) + '.csv'
        print(fname)
        ds_sub.to_csv(fname)
        fsub_count += 1
        ds_sub = pd.DataFrame()

1
0.0
1001
0.0
2001
0.0
3001
0.0
4001
0.0
5001
0.0
6001
0.0
7001
0.0
8001
0.0
9001
0.0
10001
0.0
11001
0.0
12001
0.0
13001
0.0
14001
0.0
15001
0.0
16001
0.0
17001
0.0
18001
0.0
19001
0.0
20001
0.0
21001
0.0
22001
0.0
23001
0.0
24001
0.0
25001
0.0
26001
0.0
27001
0.0
28001
0.0
29001
0.0
30001
0.0
31001
0.0
32001
0.0
33001
3.99


In [24]:
ds.to_csv('nba_comments_reg18_'+ str(fcom_count) + '.csv')
ds_sub.to_csv('nba_submissions_reg18_' + str(fsub_count) + '.csv')

In [25]:
ds

,id,author,flair,created,text,parent
0,ea7kkcq,Iblaka,Nuggets,2018-11-21 22:42:19,Kyrie so fucking woke,t3_9zafzo
1,ea7kkex,vividsang,[NYK] Allan Houston,2018-11-21 22:42:20,you just can't make certain shit up,t3_9zafzo
2,ea7kknb,marcbacus,[GSW] Draymond Green,2018-11-21 22:42:26,Fined.,t3_9zafzo
3,ea7kkpr,ShaolinCheesecake,Canada,2018-11-21 22:42:28,"He has to say ""suck my dick""",t3_9zafzo
4,ea7kktu,Spida2YBanana,[UTA] Donovan Mitchell,2018-11-21 22:42:30,I miss the old Kyrie.,t3_9zafzo
...,...,...,...,...,...,...
43649,eajuhrw,xChainfirex,Raptors,2018-11-26 23:28:09,I wish Lowry would be little better offensivel...,t3_a0o2iy
43650,eakqg83,shashyboy,None,2018-11-27 10:37:59,that last pass to kawhi just at the edge of hi...,t3_a0o2iy
43651,eakb43q,MoonBoots69,None,2018-11-27 06:55:53,The sequences of JV rebound to Kyle assist to ...,t3_a0o2iy
43652,eaj4qbu,TheLeBrontoRaptors,Lakers,2018-11-26 17:05:34,Lowry for Ingram + Ball....,t3_a0o2iy


In [26]:
ds_sub

,id,title,author,flair,created,text
0,9zafzo,[Weiss] Kyrie Irving: “Fuck Thanksgiving.”,Brad-Stevens,Celtics,2018-11-21 22:41:58,
1,9zag53,[King] After someone told him Happy Thanksgivi...,wttd40,,2018-11-21 22:42:29,
2,9zag9x,"NBA scores, highlights, updates: LeBron James ...",gamingnews500,None,2018-11-21 22:43:06,
3,9zagb2,1 Nuggets in the past 3 games; 28 ft attempts ...,dat_King,Nuggets,2018-11-21 22:43:13,Pelicans vs Nuggets (34 attempts vs 13 attempt...
4,9zagiz,Cold,colincoin472,Nets,2018-11-21 22:44:05,
...,...,...,...,...,...,...
2129,a0nzpx,New NBA Dunk Contest idea: Every team in the l...,FlatMoundOfRebound,Heat Bandwagon,2018-11-26 16:52:07,Disclaimer: This is purely hypothetical. I'm a...
2130,a0nzuo,NBA SportsLine projections: Warriors’ champion...,gamingnews500,None,2018-11-26 16:52:30,
2131,a0o0ep,[Gambo] I decided to look at the Tyson Chandle...,dasunsrule32,Suns,2018-11-26 16:54:06,
2132,a0o1mz,"I know it’s a tired topic, but another vital t...",AllAboutBucketss,Celtics,2018-11-26 16:57:53,After it’s all been pretty much been debunked ...
